In [ ]:
# import os, getpass
import chromadb
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4
from langchain_text_splitters import MarkdownTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import ChatWatsonx
# from langchain.chains import RetrievalQAjin
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from docling.document_converter import DocumentConverter
from utils import *
PAGE_BREAK = "<!-- page break -->"

In [2]:
api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
url="https://us-south.ml.cloud.ibm.com"
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"

## Load documents

In [5]:
## TODO DONE: preserve metadata e.g. page number, chunking should probably be done per page

file_path = "data/climate_edu/389801eng.pdf"
converter = DocumentConverter()
result = converter.convert(file_path)
doc = result.document


/Users/hubboko/miniconda3/envs/ibm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/hubboko/miniconda3/envs/ibm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hubboko/miniconda3/envs/ibm/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/hubboko/miniconda3/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(


In [6]:
doc_name = doc.name

In [7]:
doc_md = doc.export_to_markdown(page_break_placeholder=PAGE_BREAK, image_placeholder="",)
with open("data/climate_edu/389801eng.md", "w") as f:
    f.write(doc_md)

pages = doc_md.split(PAGE_BREAK)
pages = [markdown_cleanup(page) for page in pages]


## Chunk

In [8]:
## TODO DONE: change model to granite embed
tokenizer = AutoTokenizer.from_pretrained(
    "ibm-granite/granite-embedding-278m-multilingual"
)
max_length = tokenizer.model_max_length
chunk_overlap_length = int(max_length / 10)
chunk_size_length = max_length - chunk_overlap_length
markdown_splitter = MarkdownTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=chunk_size_length,
    chunk_overlap=int(max_length/10)
)

In [9]:
chunked_docs = []
for i, page in enumerate(pages):
    chunks = markdown_splitter.split_text(page)
    for j, chunk in enumerate(chunks):
        document = Document(
            page_content=chunk,
            embeddings=tokenizer.encode(chunk, add_special_tokens=False),
            metadata={
                "source": doc_name,
                "page": i + 1,
                "chunk": j + 1
            }
        )
        chunked_docs.append(document)

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


## Ingest

In [10]:
embedder = WatsonxEmbeddings(
    url=url,
    apikey=api_key,
    project_id=project_id,
    model_id="ibm/granite-embedding-278m-multilingual"
)

In [11]:
vector_store = Chroma(
    collection_name="climate_edu",
    persist_directory="data/chroma_db",
    embedding_function=embedder
)

In [12]:
vector_store.add_documents(chunked_docs)

['23962db0-8330-4314-a404-04367c3e9c9d',
 '77a82c22-caf8-4ac8-9232-a300c3ff70bd',
 '72c94779-9480-4765-b8a4-fd92603ca6c2',
 'de173c56-4b24-4a7d-97ba-aa09c729171b',
 'd0fb7b79-f50c-478a-ba6f-8a01e501f847',
 '77560afe-f701-4165-b040-81c7ece7afb7',
 '18e94413-484e-4e97-988f-804a3f19e2de',
 '1d42a1f7-62f0-4323-9185-0009bf7b5699',
 '0d31519f-8ff5-4ce1-81e8-a0049bdccd95',
 'bf8a0f06-ef58-40cd-b106-9ecc7f296f91',
 '70c2fbd8-d9aa-462d-85d5-84b185cfbe0e',
 '1bd84e39-5a8c-4a5c-94f6-cc16c1c1a206',
 'de23fa4c-8acb-4c8f-8236-c442301aa07f',
 'ebb7d0be-6385-4b33-b9b4-1e6db7073c57',
 'b6777b97-0625-47e3-9813-42c94d3f807b',
 '8f24417d-0603-4d53-b8f2-b52f5f5ef62b',
 '634f7c72-f103-485b-b2f6-2670920facae',
 'ffce595b-f8b0-4398-8087-f20d8e8b1f59',
 'fa23f5af-3aa3-48a8-a9ba-787cba44ab97',
 'ee4a11cf-9e17-451c-b42a-bbe49857c731',
 '340c1a69-7501-4641-9092-38a9abf18897',
 '0fd328be-0d56-4df9-a2ab-1dd32c6a3f66',
 '3475617d-d952-410f-8f68-7bb683ab3811',
 '015bd5ab-7bc3-469a-a85d-4d4f7852b681',
 'ee43d0a1-99ff-

## Retrieval

In [52]:
user_query = "What could be done by a single person to help mitigate climate change?"

In [53]:
retriever = vector_store.similarity_search(
    user_query,
    k=2,
)

## Augmentation

In [69]:
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a teacher who provide information on climate change, its impacts, and actions individuals can take to mitigate it. Use the provided context to answer each user question as accurately and briefly as possible. If the context does not contain the answer, respond with: "Sorry, I cannot assist you."
Answer the queries with the same user's input language no matter the context's language.
Always answer in markdown format, using bullet points for lists and bold for important facts. If the context is not sufficient to answer the question, politely inform the user that you cannot assist them.
Keep the the answers like you are teaching and add a little interaction to the answer, like asking a question or giving a suggestion to the user.
Example of a good answer:
<|start_header_id|>system<|end_header_id|>
Here are some actions a single person can take to help mitigate climate change:
    1. Reduce energy consumption by using energy-efficient appliances and turning off lights when not in use.
    2. Use public transportation, carpool, bike, or walk instead of driving alone.
    3. Reduce, reuse, and recycle to minimize waste.
    4. Support renewable energy sources like solar or wind power.
I hope these suggestions help you contribute to a more sustainable future! What actions do you think you can take to help mitigate climate change?
<|end_header_id|>

<|start_header_id|>user<|end_header_id|>
Context:
{retriever}

Question:
{user_query}

<|start_header_id|>assistant<|end_header_id|>
Here are some actions a single person can take to help mitigate climate change:
"""

In [70]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"
parameters = TextChatParameters(
    max_tokens = 1024,
    temperature = 0
)
watsonx_llm = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters,
    prompt_template=prompt_template
)

In [71]:

out = watsonx_llm.invoke(user_query)


In [72]:
print(out.content)

<|python_start|>actions = [
    "Reduce energy consumption by using energy-efficient appliances and turning off lights, electronics, and devices when not in use",
    "Conserve water by taking shorter showers and fixing leaks",
    "Reduce, Reuse, Recycle: minimize single-use plastics, reuse items when possible, and recycle as much as possible",
    "Use public transportation, walk, or bike when possible",
    "Eat a plant-based diet, choose locally sourced and seasonal food, and reduce food waste",
    "Support renewable energy by investing in solar panels or renewable energy credits",
    "Plant trees and support reforestation efforts",
    "Reduce air travel or offset carbon emissions from flights",
    "Support organizations working on climate change mitigation and adaptation efforts",
    "Educate yourself and others on climate change causes, effects, and solutions"
]

def suggest_actions():
    return actions

suggested_actions = suggest_actions()
print(suggested_actions)<|python